<a href="https://colab.research.google.com/github/componavt/neural_synset/blob/master/src/dataset/wikt_labels_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading labels with definitions

Source code: [Loading a custom dataset](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/videos/load_custom_dataset.ipynb#scrollTo=D2ekPOyykZDq), [video](https://www.youtube.com/watch?v=HyQgpJTkRdE).

Video: [The pipeline function](https://www.youtube.com/watch?v=tiZFewofSLM).

Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
#! pip install -U accelerate
! pip install -U transformers datasets evaluate seqeval

#! pip install datasets transformers[sentencepiece]
#! pip install torch               # required by TrainingArguments
#! pip install transformers[torch] # required by TrainingArguments

In [ ]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

In [ ]:
cat label_meaning.csv

"word"|"meaning"|"книжн."|"ирон."|"религ."|"груб."
подвизаться|осуществлять деятельность, работать, действовать в какой-нибудь области|1|1|0|0
подвизаться|совершать подвиг в чём-либо, часто о ежедневном борении|0|0|1|0
заткнуться|то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть|0|0|0|1
пустобрёх|тот, кто говорит много пустого и несерьёзного; болтун|0|0|0|1
излаять|сильно изругать|0|0|0|1
бизнес-дама|о предпринимательнице|0|1|0|0
агнец божий|кроткий, робкий, безобидный человек|0|1|0|0
всезнайка|человек, который считает себя знающим всё|0|1|0|0
галантерейный|относящийся к галантерее|0|0|0|0
галантерейный|чрезмерно любезный, вежливый до слащавости|0|1|0|0
дитятя|дитя, ребёнок, чадо|0|1|0|0


In [ ]:
from datasets import load_dataset

ds = load_dataset("csv", data_files="label_meaning.csv", sep="|")  # , split='train')
print(ds["train"][0]["word"], "=", ds["train"][0]["meaning"], )

Generating train split: 0 examples [00:00, ? examples/s]

подвизаться = осуществлять деятельность, работать, действовать в какой-нибудь области


In [ ]:
def get_training_corpus(dataset):
    d = dataset["train"]
    for i in range(0, len(d), 1000):
        samples = d[i : i + 1000]
        yield samples["meaning"]
training_corpus = get_training_corpus(ds)

In [ ]:
print(list(training_corpus))

[['осуществлять деятельность, работать, действовать в какой-нибудь области', 'совершать подвиг в чём-либо, часто о ежедневном борении', 'то же, что замолчать; перестать говорить, кричать, плакать; замолкнуть', 'тот, кто говорит много пустого и несерьёзного; болтун', 'сильно изругать', 'о предпринимательнице', 'кроткий, робкий, безобидный человек', 'человек, который считает себя знающим всё', 'относящийся к галантерее', 'чрезмерно любезный, вежливый до слащавости', 'дитя, ребёнок, чадо']]


# [Trainer-able Dataset object](https://stackoverflow.com/a/75717951/1173350)

In [ ]:
# To convert list of dict to Dataset objectimport pandas as pd
from datasets import Dataset

sentences = [
{'text': 'I live in Madrid', 'labels':['O', 'O', 'O', 'B-LOC']},
{'text': 'Peter lives in Spain', 'labels':['B-PER', 'O', 'O', 'B-LOC']},
{'text': 'He likes pasta', 'labels':['O', 'O', 'B-FOOD']},
]

ds = Dataset.from_pandas(pd.DataFrame(data=sentences))

In [ ]:
# Convert the dataset into a "Trainer-able" Dataset object
from datasets import Dataset
from datasets import ClassLabel

# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['B-LOC', 'B-PER', 'B-FOOD', 'O'])


train_sentences = [
{'text': 'I live in Madrid', 'labels':['O', 'O', 'O', 'B-LOC']},
{'text': 'Peter lives in Spain', 'labels':['B-PER', 'O', 'O', 'B-LOC']},
{'text': 'He likes pasta', 'labels':['O', 'O', 'B-FOOD']},
]

# Map text to tokenizer ids.
ds = ds.map(lambda x: tokenizer(x["text"], truncation=True))

# Map labels to label ids.
ds = ds.map(lambda y: {"labels": classmap.str2int(y["labels"])})

In [ ]:
# To compute metrics with the labeled inputs that you have:
import evaluate

metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# To use with the Trainer object

import pandas as pd
import evaluate

from datasets import Dataset
from datasets import ClassLabel

from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification

# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['B-LOC', 'B-PER', 'B-FOOD', 'O'])

train_sentences = [
{'text': 'I live in Madrid', 'labels':['O', 'O', 'O', 'B-LOC']},
{'text': 'Peter lives in Spain', 'labels':['B-PER', 'O', 'O', 'B-LOC']},
{'text': 'He likes pasta', 'labels':['O', 'O', 'B-FOOD']},
]

eval_sentences = [
    {"text": "I like pasta from Madrid , Spain", 'labels': ['O', 'O', 'B-FOOD', 'O', 'B-LOC', 'O', 'B-LOC']}
]

ds_train = Dataset.from_pandas(pd.DataFrame(data=train_sentences))
ds_eval = Dataset.from_pandas(pd.DataFrame(data=eval_sentences))

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-multilingual-cased",
                                                        id2label={i:classmap.int2str(i) for i in range(classmap.num_classes)},
                                                        label2id={c:classmap.str2int(c) for c in classmap.names},
                                                        finetuning_task="ner")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
data_collator = DataCollatorForTokenClassification(tokenizer)


ds_train = ds_train.map(lambda x: tokenizer(x["text"], truncation=True))
ds_eval = ds_eval.map(lambda x: tokenizer(x["text"], truncation=True))

ds_train = ds_train.map(lambda y: {"labels": classmap.str2int(y["labels"])})
ds_eval = ds_eval.map(lambda y: {"labels": classmap.str2int(y["labels"])})


metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize our Trainer
trainer = Trainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

# AutoTokenizer and PyTorch optimized training loop
From [quicktour.ipynb#AutoTokenizer](https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/quicktour.ipynb#scrollTo=c-mB_1hXw57y&line=1&uniqifier=1)

In [ ]:
# setup hyperparameters (learning rate, batch size, and the number of epochs to train for)
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./pt_training",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    )

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model_name2 = "Babelscape/wikineural-multilingual-ner"     # see https://huggingface.co/Babelscape/wikineural-multilingual-ner
model_name3 = "orgcatorg/bert-base-multilingual-cased-ner" # see https://huggingface.co/orgcatorg/bert-base-multilingual-cased-ner
pt_model    = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer   = AutoTokenizer.from_pretrained(model_name)
tokenizer2  = AutoTokenizer.from_pretrained(model_name2)
tokenizer3  = AutoTokenizer.from_pretrained(model_name3)

In [ ]:
tokenizer_new  = tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer_new2 = tokenizer2.train_new_from_iterator(training_corpus, 52000)
tokenizer_new3 = tokenizer3.train_new_from_iterator(training_corpus, 52000)

In [ ]:
# ds["train"][0]["meaning"]

In [ ]:
s = ds["train"][0]["meaning"]
print(s)
#tokens_old = old_tokenizer.tokenize(s)
print(tokenizer(s),  "tokens by MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
print(tokenizer2(s), "tokens by Babelscape/wikineural-multilingual-ner")     # subword tokenization
print(tokenizer3(s), "tokens by orgcatorg/bert-base-multilingual-cased-ner") # subword tokenization

#tokens = tokenizer.tokenize( ds["train"][0]["meaning"] )
#print("tokens=", tokens)

осуществлять деятельность, работать, действовать в какой-нибудь области
{'input_ids': [1, 24527, 22857, 73608, 834, 262, 260, 28877, 262, 19384, 16877, 316, 1067, 818, 265, 669, 14320, 9407, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} tokens by MoritzLaurer/mDeBERTa-v3-base-mnli-xnli
{'input_ids': [101, 85854, 10227, 92157, 32589, 34112, 117, 45680, 117, 105805, 76183, 543, 78351, 118, 19544, 91369, 11431, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} tokens by Babelscape/wikineural-multilingual-ner
{'input_ids': [101, 85854, 10227, 92157, 32589, 34112, 117, 45680, 117, 105805, 76183, 543, 78351, 118, 19544, 91369, 11431, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
def tokenize_dataset(dataset):
  return tokenizer(dataset["meaning"])

In [ ]:
ds["train"]
#ds

In [ ]:
#dataset = ds["train"].map(tokenize_dataset, batched=True)
dataset = ds.map(tokenize_dataset, batched=True)
dataset

In [ ]:
# create a batch of examples from dataset
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

In [ ]:
# 80% train, 20% test + validation
dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True)
dataset

In [ ]:
# gather all these classes in Trainer:
from transformers import Trainer
trainer = Trainer(
    model=pt_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

#see https://discuss.huggingface.co/t/the-model-did-not-return-a-loss-from-the-inputs-only-the-following-keys-logits-for-reference-the-inputs-it-received-are-input-values/25420